<a href="https://colab.research.google.com/github/LucasFerrSil/GrafosDoBitcoin/blob/main/CR_Blockchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

hash = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/HashBlocks.xlsx')
hashBlocks = list(hash['Hash'])

api_url = [('https://blockchain.info/rawblock/' + str(hashBlocks[i])) for i in range(len(hashBlocks))]
response = [requests.get(api_url[i]).json() for i in range(len(api_url))]
tx_data = [response[i]['tx'] for i in range(len(response))]

Mounted at /content/drive


In [ ]:
transacao = []
for i in range(len(tx_data)):
  vetor = []
  for j in range(len(tx_data[i])):
    output = []
    input = []
    for k in range(len(tx_data[i][j]['inputs'])):
      if(j != 0 and tx_data[i][j]['inputs'][k]['prev_out']['spent'] == True):
        input.append([tx_data[i][j]['inputs'][k]['prev_out']['addr'],tx_data[i][j]['inputs'][k]['prev_out']['value']])
      else:
        input.append(['Inicial',0.0000000000000000001])
    for w in range(len(tx_data[i][j]['out'])):
      if(tx_data[i][j]['out'][w]['spent'] == True):
        output.append([tx_data[i][j]['out'][w]['addr'],tx_data[i][j]['out'][w]['value']])
    vetor.append([input,output])
  transacao.append(vetor)

In [ ]:
def arestas(transacao):
  inputs = transacao[0]
  outputs = transacao[1]
  valores_inputs = [inputs[i][1] for i in range(len(inputs))]
  valores_outputs = [outputs[i][1] for i in range(len(outputs))]
  soma_input = sum(valores_inputs)
  lista = []
  edges = []
  for i in range(len(inputs)):
    for j in range(len(outputs)):
      k = valores_outputs[j]/soma_input
      lista.append(k*valores_inputs[i])
      edges.append([inputs[i][0],outputs[j][0]])
  for i in range(len(lista)):
    edges[i].append(lista[i])
  return edges

edges = []
for i in range(len(transacao)):
  bloco = []
  for j in range(len(transacao[i])):
    x = arestas(transacao[i][j])
    for combinacao in x:
      bloco.append(combinacao)
  edges.append(bloco)


DataFrames = [pd.DataFrame(edges[i],columns=['Source','Target','Weight']) for i in range(len(edges))]
i = 1
for df in DataFrames:
  df.to_csv(f'/content/drive/MyDrive/Nova pasta/Edges-2013-0{i}.csv',index=False)
  i += 1

In [ ]:
lista_edges = []
for df in df2:
  input = [df['Inputs'][i] for i in range(len(df['Inputs']))]
  output = [[df['Outputs'][i],df['Valor'][i]] for i in range(len(df['Inputs']))]
  copyinput = input
  copyoutput = output
  lista = []
  i = 0
  while i != len(copyinput):
    indices = []
    termo = []
    input_analisado = copyinput[i]
    termo.append(copyinput[i])
    termo.append(copyoutput[i])
    for k in range(len(copyinput)):
      if(k != i and copyinput[k]==copyinput[i]):
        indices.append(k)
    for indice in indices:
      termo.append(output[indice])
      copyinput.remove(input_analisado)
    lista.append(termo)
    i += 1
  edges = []
  for i in range(len(lista)):
    for j in range(1,len(lista[i])):
      termo = (lista[i][0],lista[i][j][0],lista[i][j][1])
      edges.append(termo)
  lista_edges.append(edges)

In [ ]:
for i in range(len(tx_data)):
  for j in range(len(tx_data[i])):
    for w in range(len(tx_data[i][j]['out'])):
      if((tx_data[i][j]['out'][w]['spent']) == False and (tx_data[i][j]['out'][w]['value'] != 0)):
        tx_data[i][j]['out'][w]['spent'] = True

Criação de grafos resumo

In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Para os grafos representando o ano inteiro, fazer função distribuição discreta para o grau de cada input e de cada output
#dps criar grafos com um número consideravel de arestas e nós e buscar a lei de potencia

def distribuicao(lista):
  minl = min(lista)
  maxl = max(lista)
  n = int(len(lista)**(1/2))
  inter = ((maxl-minl)/n)
  x = []
  for i in range(n):
    v = 0
    for j in range(len(lista)):
      if(minl+inter*i <= lista[j] and lista[j] <= (minl+inter*(i+1))):
        v += 1
    x.append(v)
  soma = sum(x)
  f = [x[i]/soma for i in range(len(x))]
  intervalo = [[minl+inter*i,minl+inter*(i+1)] for i in range(n)]
  return intervalo,f

In [ ]:
arestas_2023 = [pd.read_csv(f'/content/drive/My Drive/CR/Edges-2023-0{i}.csv', sep=";") if (i not in [7,8]) else pd.read_csv(f'/content/drive/My Drive/CR/Edges-2023-0{i}.csv', sep=",") for i in range(1,11)]
arestas_2022 = [pd.read_csv(f'/content/drive/My Drive/CR/Edges-2022-0{i}.csv', sep=";") if (i not in [8]) else pd.read_csv(f'/content/drive/My Drive/CR/Edges-2022-0{i}.csv', sep=",") for i in range(1,13)]
arestas_2021 = [pd.read_csv(f'/content/drive/My Drive/CR/Edges-2021-0{i}.csv', sep=";") for i in range(1,13)]
arestas_2013 = [pd.read_csv(f'/content/drive/My Drive/CR/Edges-2013-0{i}.csv', sep=";") for i in range(1,13)]

In [ ]:

valores2023 = []
info_inputs2023 = []
info_outputs2023 = []

for i in range(len(arestas_2023)):
  mes = arestas_2023[i]
  x = list(mes['Weight'])
  for valor in x:
    valores2023.append(valor)
  y = list(mes['Source'])
  for input in y:
    info_inputs2023.append(input)
  z = list(mes['Target'])
  for output in z:
    info_outputs2023.append(output)


valores2022 = []
info_inputs2022 = []
info_outputs2022 = []

for i in range(len(arestas_2022)):
  mes = arestas_2022[i]
  x = list(mes['Weight'])
  for valor in x:
    valores2022.append(valor)
  y = list(mes['Source'])
  for input in y:
    info_inputs2022.append(input)
  z = list(mes['Target'])
  for output in z:
    info_outputs2022.append(output)


valores2021 = []
info_inputs2021 = []
info_outputs2021 = []

for i in range(len(arestas_2021)):
  mes = arestas_2021[i]
  x = list(mes['Weight'])
  for valor in x:
    valores2021.append(valor)
  y = list(mes['Source'])
  for input in y:
    info_inputs2021.append(input)
  z = list(mes['Target'])
  for output in z:
    info_outputs2021.append(output)


valores2013 = []
info_inputs2013 = []
info_outputs2013 = []

for i in range(len(arestas_2013)):
  mes = arestas_2013[i]
  x = list(mes['Weight'])
  for valor in x:
    valores2013.append(valor)
  y = list(mes['Source'])
  for input in y:
    info_inputs2013.append(input)
  z = list(mes['Target'])
  for output in z:
    info_outputs2013.append(output)

In [ ]:
def distribuicao_nominal(lista):
  passados = []
  distribuicao = []
  i = 0
  while i != len(lista):
    removidos = []
    count = 1
    j = 0
    while j != (len(lista)):
      if(lista[i] not in passados and lista[i] == lista[j] and i != j):
        count += 1
        lista.pop(j)
      else:
        j += 1
    distribuicao.append(count)
    i += 1
  return distribuicao

In [ ]:
def inteirar(lista):
  for i in range(len(lista)):
    if(type(lista[i]) == str):
      num = ''
      for x in lista[i]:
        if(x != '.'):
          num += x
      lista[i] = float(num)

In [ ]:
dis_valores2023 = distribuicao(valores2023)
inteirar(valores2022)
dis_valores2022 = distribuicao(valores2022)
dis_valores2021 = distribuicao(valores2021)
dis_valores2013 = distribuicao(valores2013)

In [ ]:
dis_valores2013

([[0.028012713, 4954954954.982904],
  [4954954954.982904, 9909909909.937798],
  [9909909909.937798, 14864864864.89269],
  [14864864864.89269, 19819819819.84758],
  [19819819819.84758, 24774774774.80247],
  [24774774774.80247, 29729729729.757366],
  [29729729729.757366, 34684684684.71226],
  [34684684684.71226, 39639639639.66715],
  [39639639639.66715, 44594594594.62205],
  [44594594594.62205, 49549549549.576935],
  [49549549549.576935, 54504504504.53183],
  [54504504504.53183, 59459459459.486725],
  [59459459459.486725, 64414414414.44161],
  [64414414414.44161, 69369369369.3965],
  [69369369369.3965, 74324324324.3514],
  [74324324324.3514, 79279279279.30629],
  [79279279279.30629, 84234234234.26118],
  [84234234234.26118, 89189189189.21608],
  [89189189189.21608, 94144144144.17096],
  [94144144144.17096, 99099099099.12585],
  [99099099099.12585, 104054054054.08075],
  [104054054054.08075, 109009009009.03564],
  [109009009009.03564, 113963963963.99054],
  [113963963963.99054, 1189189189

In [ ]:
dis_info_inputs2023 = distribuicao_nominal(info_inputs2023)
dis_info_outputs2023 = distribuicao_nominal(info_outputs2023)

dis_info_inputs2022 = distribuicao_nominal(info_inputs2022)
dis_info_outputs2022 = distribuicao_nominal(info_outputs2022)

dis_info_inputs2021 = distribuicao_nominal(info_inputs2021)
dis_info_outputs2021 = distribuicao_nominal(info_outputs2021)

dis_info_inputs2013 = distribuicao_nominal(info_inputs2013)
dis_info_outputs2013 = distribuicao_nominal(info_outputs2013)